In [2]:
import pandas as pd
import pandasql as ps

In [46]:
pathPedido = '../data/pedido.csv'
pathItem = '../data/item.csv'

In [42]:
pedido = pd.read_csv(pathPedido)

In [43]:
pedido.head()

,idPedido,idCliente,descSituacao,dtPedido,dtAprovado,dtEnvio,dtEntrega,dtEntregaEstimada
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02,2017-10-02,2017-10-04,2017-10-10,2017-10-18
1,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18,2017-11-18,2017-11-22,2017-12-02,2017-12-15
2,a4591c265e18cb1dcee52889e2d8acc3,503740e9ca751ccdda7ba28e9ab8f608,delivered,2017-07-09,2017-07-09,2017-07-11,2017-07-26,2017-08-01
3,e69bfb5eb88e0ed6a785585b27e16dbf,31ad1d1b63eb9962463f764d4e6e0c9d,delivered,2017-07-29,2017-07-29,2017-08-10,2017-08-16,2017-08-23
4,34513ce0c4fab462a55830c0989c7edb,7711cf624183d843aafe81855097bc37,delivered,2017-07-13,2017-07-13,2017-07-14,2017-07-19,2017-08-08


In [47]:
item = pd.read_csv(pathItem)

In [48]:
item.head()

,idPedido,idPedidoItem,idProduto,idVendedor,shipping_limit_date,vlPreco,vlFrete
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [57]:
tb_pedido_item = ps.sqldf(""" select
                                    t2.*,
                                    t1.dtPedido
                              from pedido as t1

                              left join item as t2
                              on t1.idPedido = t2.idPedido

                              WHERE t1.dtPedido < '2018-01-01'
                              and t1.dtPedido >= '2017-07-01'
                              and t2.idVendedor is not null
                """)

In [58]:
tb_pedido_item.head()

,idPedido,idPedidoItem,idProduto,idVendedor,shipping_limit_date,vlPreco,vlFrete,dtPedido
0,e481f51cbdc54678b7cc49136f2d6af7,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,2017-10-02
1,949d5b44dbf5de918fe9c16f97b45f8a,1,d0b61bfb1de832b15ba9d266ca96e5b0,66922902710d126a0e7d26b0e3805106,2017-11-23 19:45:59,45.00,27.20,2017-11-18
2,a4591c265e18cb1dcee52889e2d8acc3,1,060cb19345d90064d1015407193c233d,8581055ce74af1daba164fdbd55a40de,2017-07-13 22:10:13,147.90,27.36,2017-07-09
3,e69bfb5eb88e0ed6a785585b27e16dbf,1,9a78fb9862b10749a117f7fc3c31f051,7c67e1448b00f6e969d365cea6b010ab,2017-08-11 12:05:32,149.99,19.77,2017-07-29
4,34513ce0c4fab462a55830c0989c7edb,1,f7e0fa615b386bc9a8b9eb52bc1fff76,87142160b41353c4e5fca2360caf6f92,2017-07-19 20:10:08,98.00,16.13,2017-07-13


In [72]:
tb_summary = ps.sqldf("""
                    select idVendedor,
                    count(distinct idPedido) as qtdPedidos,
                    count(distinct date(dtPedido)) as qtdDias,
                    count(idProduto) as qtdItens,
                    (JulianDay('2018-01-01') - JulianDay(max(dtPedido))) as qtdRecencia,
                    sum(vlPreco) / count(distinct idPedido) as avgTicket,
                    avg(vlPreco) as avgValorProduto,
                    max(vlPreco) as maxValorProduto,
                    min(vlPreco) as minValorProduto,
                    count(idProduto) / count(distinct idPedido) as avgProdutoPedido
                    
                    from tb_pedido_item      
                    
                    group by idVendedor
""")

In [73]:
tb_summary.head()

,idVendedor,qtdPedidos,qtdDias,qtdItens,qtdRecencia,avgTicket,avgValorProduto,maxValorProduto,minValorProduto,avgProdutoPedido
0,0015a82c2db000af6aaaf3ae2ecb0532,3,3,3,75.0,895.000000,895.000000,895.0,895.0,1
1,001cca7ae9ae17fb1caed9dfb1094831,98,68,116,5.0,121.815816,102.913362,129.0,79.0,1
2,002100f778ceb8431b7a1020ff7ab48f,39,30,43,4.0,20.505128,18.597674,49.9,9.9,1
3,003554e2dce176b5555353e4f3555ac8,1,1,1,17.0,120.000000,120.000000,120.0,120.0,1
4,004c9cd9d87a3c30c522c48c4fc07416,84,66,87,9.0,111.260476,107.423908,233.8,47.9,1


In [74]:
tb_pedido_summary = ps.sqldf("""
                        select 
                            idVendedor,
                            idPedido,
                            sum(vlPreco) as vlPreco
                            
                        from tb_pedido_item
                        
                        group by idVendedor, idPedido
                            

                """)

In [75]:
tb_pedido_summary.head()

,idVendedor,idPedido,vlPreco
0,0015a82c2db000af6aaaf3ae2ecb0532,7f39ba4c9052be115350065d07583cac,895.0
1,0015a82c2db000af6aaaf3ae2ecb0532,9dc8d1a6f16f1b89874c29c9d8d30447,895.0
2,0015a82c2db000af6aaaf3ae2ecb0532,d455a8cb295653b55abda06d434ab492,895.0
3,001cca7ae9ae17fb1caed9dfb1094831,038345c4a9d21c17180f2f77e1701c69,99.0
4,001cca7ae9ae17fb1caed9dfb1094831,06c07d5b2696b9631375e2877c955b3b,99.0


In [81]:
tb_min_max = ps.sqldf("""
                    select 
                        idVendedor,
                        max(vlPreco) as maxVlPedido,
                        min(vlPreco) as minVlPedido
                        
                    from tb_pedido_summary
                    
                    group by idVendedor
""")

In [82]:
tb_min_max.head()

,idVendedor,maxVlPedido,minVlPedido
0,0015a82c2db000af6aaaf3ae2ecb0532,895.0,895.0
1,001cca7ae9ae17fb1caed9dfb1094831,297.0,79.0
2,002100f778ceb8431b7a1020ff7ab48f,49.9,9.9
3,003554e2dce176b5555353e4f3555ac8,120.0,120.0
4,004c9cd9d87a3c30c522c48c4fc07416,233.8,47.9


In [90]:
tb_life = ps.sqldf(""" 
                              select
                                    t2.idVendedor,
                                    sum(vlPreco) as  LTV,
                                    ('2918-01-01' -max(dtPedido)) as qtdDiasBase,
                                    t1.dtPedido
                              from pedido as t1

                              left join item as t2
                              on t1.idPedido = t2.idPedido

                              WHERE t1.dtPedido < '2018-01-01'
                              and t2.idVendedor is not null
                              
                              group by t2.idVendedor
                """)

In [91]:
tb_life.head()

,idVendedor,LTV,qtdDiasBase,dtPedido
0,0015a82c2db000af6aaaf3ae2ecb0532,2685.00,901,2017-10-18
1,001cca7ae9ae17fb1caed9dfb1094831,11937.95,901,2017-12-27
2,002100f778ceb8431b7a1020ff7ab48f,799.70,901,2017-12-28
3,003554e2dce176b5555353e4f3555ac8,120.00,901,2017-12-15
4,004c9cd9d87a3c30c522c48c4fc07416,9345.88,901,2017-12-23
